<a href="https://colab.research.google.com/github/comojin1994/Deep_Learning_Study/blob/master/AI_Competition/EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import PIL
from PIL import Image, ImageDraw
import xml.etree.ElementTree as elemTree
import cv2
import torch
import random
%matplotlib inline

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.dterministic = True
    torch.backends.cudnn.benchmark = True

SEED = 42
seed_everything(SEED)

In [ ]:
os.getcwd()

In [ ]:
os.listdir(f'../../../datasets/objstrgzip/07_object_detection/07_object_detection')

In [ ]:
BASE_DIR = f'../../../datasets/objstrgzip/07_object_detection/07_object_detection'
train_dir = f'{BASE_DIR}/train/'
val_dir = f'{BASE_DIR}/val/'
test_dir = f'{BASE_DIR}/test/'

In [ ]:
train_df = elemTree.parse(f'{BASE_DIR}/train/train.xml')
root = train_df.getroot()

In [ ]:
sample_image = root.find('image')
print(sample_image.attrib)
for box in iter(sample_image.findall('box')):
    print(box.attrib)

In [ ]:
img = cv2.imread(os.path.join(train_dir, sample_image.attrib['name']))
plt.imshow(img)
plt.show()

In [ ]:
def change_type(x):
    return int(float(x))

for box in iter(sample_image.findall('box')):
    x = (change_type(box.attrib['xtl']), change_type(box.attrib['ytl']))
    y = (change_type(box.attrib['xbr']), change_type(box.attrib['ybr']))
    cv2.rectangle(img, x, y, (255, 0, 0), 4)
plt.imshow(img)
plt.show()

In [ ]:
img_name, height, width, label, x1, y1, x2, y2= [], [], [], [], [], [], [], []
box_w, box_h, area = [], [], []
for info in root.findall('image'):
    for box in info.findall('box'):
        img_name.append(info.attrib['name'])
        height.append(int(info.attrib['height']))
        width.append(int(info.attrib['width']))
        label.append(box.attrib['label'])
        x1.append(change_type(box.attrib['xtl']))
        y1.append(change_type(box.attrib['ytl']))
        x2.append(change_type(box.attrib['xbr']))
        y2.append(change_type(box.attrib['ybr']))
        box_w.append(change_type(box.attrib['xbr']) - change_type(box.attrib['xtl']))
        box_h.append(change_type(box.attrib['ybr']) - change_type(box.attrib['ytl']))
        bw = change_type(box.attrib['xbr']) - change_type(box.attrib['xtl'])
        bh = change_type(box.attrib['ybr']) - change_type(box.attrib['ytl'])
        area.append(bw * bh)

image_info = pd.DataFrame({'name': img_name, 'height': np.array(height),
                           'width': np.array(width), 'label': np.array(label),
                          'x1': np.array(x1), 'y1': np.array(y1),
                          'x2': np.array(x2), 'y2': np.array(y2),
                          'box_w': np.array(box_w), 'box_h': np.array(box_h),
                          'area': np.array(area)})

In [ ]:
image_info.head()

In [ ]:
image_info.dtypes

In [ ]:
image_info.groupby('label').size()

In [ ]:
pd.DataFrame(image_info[['box_w', 'box_h']]).boxplot()
plt.show()

In [ ]:
image_info[image_info['box_w'] == max(image_info['box_w'])]

In [ ]:
img = cv2.imread(f'{train_dir}/MP_SEL_026237.jpg')
sample_image2 = image_info[image_info['name']=='MP_SEL_026237.jpg'].loc[:, ['x1', 'y1', 'x2', 'y2']]

for x1, y1, x2, y2 in np.array(sample_image2):
    x = (x1, y1)
    y = (x2, y2)
    cv2.rectangle(img, x, y, (255, 0, 0), 4)
plt.imshow(img)
plt.show()